In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tokenization
import collections
import ast

In [2]:
columns = ['character1','character2','movietitle','conversation']

movie_conver = []
with open('data/movie_conversations.txt','r') as file:
    for l in file:
        lines = l.split('+++$+++')
        line_content =  eval(lines[-1].rstrip())
        lines = [x.strip() for x in lines[:-1]]
        lines.append(line_content)
        movie_conver.append(lines)
movie_conver = pd.DataFrame(movie_conver, columns=columns)

In [3]:
columns = ['line','character_id','movietitle','character_name','text']

movie_lines = []
with open('data/movie_lines.txt','r', errors='ignore') as file:
    for l in file:
        lines = l.split('+++$+++')
        lines[-1] = lines[-1].rstrip()
        lines = [x.strip() for x in lines]
        movie_lines.append(lines)
movie_lines = pd.DataFrame(movie_lines, columns=columns)

In [4]:
assert len(movie_lines['line'].unique()) == len(movie_lines['line'])
movie_lines.set_index('line',inplace=True)

In [7]:
class Example():
    
    def __init__(self,input_sentences, output_sentences):
        
        self.input_sentences = input_sentences
        self.output_sentences = output_sentences
               

In [ ]:
def convert_toIndex(tokens, word_dict):

    index_features = []
    for token in tokens:
        if not token in word_dict:
            word_dict[token] = len(word_dict) + 1

        index_features.append(word_dict[token])
    return index_features

In [ ]:
def convert_toExample(num_i, conversation, tokenizer, max_input_seq, max_output_seq, word_dict):
#     convert text into examples
    
    num_convers = len(conversation)
    for i, _ in enumerate(conversation):
        
        if i < num_convers - 2:
            num_i += 1
            
            text_q = tokenizer.tokenize(conversation[i])
            text_p = tokenizer.tokenize(conversation[i+1])
            response = tokenizer.tokenize(conversation[i+2])
            
            input_texts = text_q + '[SEQ]' + text_p
            input_id = convert_toIndex(input_texts, word_dict)
            output_id = convert_toIndex(response, word_dict)
            
            if len(input_id) < max_input_seq:
                input_id.extend([0] * (max_input_seq - len(input_id)))
                
            if len(output_id) < max_output_seq:
                output_id.extend([0] * (max_output_seq - len(output_id)))
                
            input_id = input_id[:max_input_seq]
            output_id = output_id[:max_output_seq]
            
            assert len(input_id) == max_input_seq
            assert len(output_id) == max_output_seq
            
            if num_i % 500 == 0:
                tf.logging.info('input text: %s' % ' '.join(input_texts))
                tf.logging.info('input text: %s' % ' '.join(input_texts))
                tf.logging.info('input text: %s' % ' '.join(input_texts))
                tf.logging.info('input text: %s' % ' '.join(input_texts))
            
            yield num_i, Example(input_id, output_id)
        

In [18]:
def build_dataSet(movie_conver, movie_lines, max_input_seq, max_output_seq, tokenizer):
    
    data = []
    word_dict = collections.OrderedDict()
    num_i = 0
    
    for index, row in movie_conver.iterrows():
        line_index = row['conversation']
        conversation = movie_lines.loc[line_index,'text']
        
        if len(conversation) > 2:
            for num_i, example in convert_toExample(num_i, conversation, tokenizer, max_input_seq, max_output_seq, word_dict):
    #             input_texts, response, input_id, output_id = text_info
                data.append(Example(input_id, output_id))
        
    

In [20]:
tokenizer = tokenization.WordpieceTokenizer()
# build_dataSet(movie_conver, movie_lines)

TypeError: __init__() missing 1 required positional argument: 'vocab'